In [1]:
import os
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, ToolMessage, AIMessage, SystemMessage
from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, START, END
from langchain_core.tools import tool
from typing import TypedDict, Annotated
import operator

# Set your Serper tool for web search
SERPER_API_KEY = os.getenv("SERPER_API_KEY")

# Initialize the Ollama model and bind the tool
# llm = ChatOpenAI(model="gpt-4o")
llm = ChatOllama(model="qwen3", temperature=0)

In [2]:
def read_md_file(filename: str) -> str:
    """Read a markdown file from the 'config' folder and return its content as a string."""
    config_path = os.path.join("config", filename)
    with open(config_path, "r", encoding="utf-8") as f:
        return f.read()
    
def format_message(message_object) -> str:
    """
    Converts a LangChain message object into a human-readable string.

    Args:
        message_object: An instance of AIMessage, HumanMessage, or ToolMessage.

    Returns:
        A formatted string with a prefix (System:, User:, Tool:).
    """
    if isinstance(message_object, AIMessage):
        return f"System: {message_object.content}"
    elif isinstance(message_object, HumanMessage) or isinstance(message_object, ToolMessage):
        return f"User: {message_object.content}"
    else:
        # Handle any other message types gracefully
        return f"Unknown: {str(message_object.content)}"
    
# Define the state of our graph
class State(TypedDict):
    system_prompt: str
    formatted_messages: list[str]
    messages: Annotated[list, operator.add]

# --- Tool Section ---

# This tool runs when called by the model.
@tool()
def human_tool(query: str):
    """Ask the human user a question when you need information that cannot be found via search.
    
    Use this to ask for:
    - Package weight (if not searchable)
    - Destination ZIP code (if they only provided a city name and search didn't help)
    - Any clarification questions
    
    Args:
        query: The specific question to ask the user
        
    Returns:
        The user's response as a string
    """
    print(f"\n🤖 AI is requesting Human assistance for: '{query}'")
    human_input = input("🧑‍💻 Your response: ")
    print (f"🧑‍💻 Human provided: '{human_input}'\n")
    return human_input


@tool
def google_search(query: str, num: int = 3):
    """Search Google for information. Use this to look up ZIP codes or item weights.
    
    Examples of good queries:
    - "ZIP code for Albany New York"
    - "MacBook Pro 15 weight in pounds"
    - "average laptop weight"
    
    Args:
        query: The search query string
        num: Number of results to return (default 3)
        
    Returns:
        Search results as a formatted string
    """
    url = "https://google.serper.dev/search"
    headers = {"X-API-KEY": SERPER_API_KEY, "Content-Type": "application/json"}
    params = {"q": query, "num" : num}
    
    print(f"Performing Google search for: '{query}'")
    
    response = requests.post(url, json=params, headers=headers)
    
    if response.status_code == 200:
        results = response.json()
        return results.get("organic", [])  # Extract search results
    else:
        return f"Error: {response.status_code}, {response.text}"
    
# Tool: shipping_rate_lookup
# shipping_rate_lookup` tool that accepts `weight` and `destination_zip` and returns a shipping rate. For now it 
# # returns a fixed rate of $50. In a real implementation, this would query a database or external service.
@tool
def shipping_rate_lookup(weight: float, destination_zip: str) -> dict:
    """Calculate shipping rate for a package.
    
    Only call this tool after you have confirmed BOTH parameters with the user.
    
    Args:
        weight: Package weight in pounds (must be a positive number)
        destination_zip: 5-digit US ZIP code (as a string)
        
    Returns:
        Dictionary with rate information including price, currency, weight, and destination
    """
    print(f"Looking up shipping rate for weight: {weight} lbs to zip: {destination_zip}")
    
    try:
        w = float(weight)
    except Exception:
        return {"error": "Invalid weight provided"}

    # In a real implementation you'd query a DB or service. Here return fixed rate.
    rate = 50.0
    return {"rate": rate, "currency": "USD", "weight": w, "destination_zip": str(destination_zip)}

tools = [human_tool, google_search, shipping_rate_lookup]
model = llm.bind_tools(tools)

tool_node = ToolNode(tools)

In [3]:
def router(state: State):
    """Router to decide the next step."""
    print("-------Inside AI router. ------")
    messages = state.get('messages', [])
    if not messages:
        # No messages, perhaps end or handle as an error
        return END

    last_message = messages[-1]

    # If the last message has tool calls, route to the tool node
    if getattr(last_message, 'tool_calls', None):
        return "call_tools"
    
    # Otherwise, you can decide to continue the conversation or end.
    # Here, we'll just end if there are no tool calls.
    return END

In [4]:
# --- GRAPH NODES ---
def chatbot(state: State):
    """The chatbot node that calls the LLM."""
    print(100*"=")
    
    # This print statement is now safe and handles any message type with a .content attribute.
    # print(f"---🤖 Chatbot thinking... Reacting to message of type: {type(last_message).__name__}---")
    print(f"---🤖 Chatbot thinking... Reacting to message of type: {state["formatted_messages"]}--")

    # The graph state `messages` already contains the full history.
    # We just need to add the system prompt for this specific invocation.
    # messages_for_llm = [SystemMessage(content=state["system_prompt"])] + state["messages"]
    messages_for_llm = [SystemMessage(content=state["system_prompt"])] + state["messages"]
    print(100*"-")
    
    response = model.invoke(messages_for_llm)  
    
    print(f"LLM Response: {response}")
    print(100*"=")
        
    return {"messages": [response]}

In [5]:
# --- GRAPH DEFINITION ---
graph = StateGraph(State)

graph.add_node("chatbot", chatbot)
graph.add_node("call_tools", tool_node)

# Set the entry point and edges
graph.set_entry_point("chatbot")
graph.add_conditional_edges("chatbot", 
                            router, 
                            {"call_tools": "call_tools", 
                             END: END} 
                            )
graph.add_edge("call_tools", "chatbot")

# Compile the graph
memory = MemorySaver()
app = graph.compile(checkpointer=memory)

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [6]:
# --- UI DRIVER ---
# This loop is still necessary to start each new conversation turn.
print("Chatbot is ready. Type 'quit' to exit.")

#I want to ship a computer to new york
#How much is tio ship a Macbook Prom m1 to New York?
#How much is to ship to New York?

config = {"configurable": {"thread_id": "thread_123"}}

while True:
    user_input = input("👤 You: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break

    # This kicks off the graph execution for the current turn
    for chunk in app.stream(
        {"messages": [user_input],
         "formatted_messages": [f"User: {user_input}"],
         "system_prompt": read_md_file("shipping_rate_lookup.md")},
        config=config,
    ):
        # Print the final output from the chatbot after the whole loop is done
        if END in chunk:
            final_message = chunk[END]['messages'][-1]
            if final_message.content:
                print(f"🤖 AI: {final_message.content}")

Chatbot is ready. Type 'quit' to exit.
---🤖 Chatbot thinking... Reacting to message of type: ['User: How much is tio ship a Macbook Prom m1 to New York?']--
----------------------------------------------------------------------------------------------------
---🤖 Chatbot thinking... Reacting to message of type: ['User: How much is tio ship a Macbook Prom m1 to New York?']--
----------------------------------------------------------------------------------------------------
LLM Response: content='<think>\nOkay, the user wants to know the shipping rate for a MacBook Pro M1 to New York. Let me break down what I need.\n\nFirst, the tools available are human_tool, google_search, and shipping_rate_lookup. The mandatory info required is weight and destination_zip. \n\nThe user mentioned "MacBook Pro M1" as the item. I need to find its weight. Since it\'s a specific model, I can use google_search to find the weight. The query should be something like "MacBook Pro M1 weight in pounds" or similar